In [ ]:
\[media pointer="file-service://file-SutTCoXFT5hM91rfWZVLtg"]
\[media pointer="file-service://file-53Yv6NZtadmcf5Hex5CtK1"]
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from qiskit import QuantumCircuit, transpile
from qiskit\_aer import Aer
from qiskit\_aer.noise import NoiseModel, thermal\_relaxation\_error
from scipy.stats import entropy

# --- 실험 설정 ---

shots = 1024
steps = 30
initial\_n = 2
delay\_time = 30000  # ns
T1 = 50000
T2 = 30000

# --- 노이즈 모델 구성 ---

noise\_model = NoiseModel()
sim = Aer.get\_backend("aer\_simulator")
sim.set\_options(method="density\_matrix")
for q in range(20):
error = thermal\_relaxation\_error(T1, T2, delay\_time)
noise\_model.add\_quantum\_error(error, "delay", \[q])

# --- 얽힘 포함 회로 생성 함수 ---

def build\_circuit(U):
n = len(U)
qc = QuantumCircuit(n, n)
for i, theta in enumerate(U):
qc.ry(theta, i)
qc.delay(delay\_time, i, unit='ns')
for i in range(n - 1):
qc.cx(i, i + 1)
qc.barrier()
qc.measure(range(n), range(n))
return qc

# --- 피드백 구조 ---

U = \[np.random.uniform(0, np.pi) for \_ in range(initial\_n)]
entropy\_traj, bias\_traj, n\_history = \[], \[], \[]
entropy\_window, bias\_window = \[], \[]

# 조건 설정

entropy\_threshold = 0.05  # 엔트로피 변화량이 이보다 작을 경우
bias\_threshold = 0.1      # 출력 편향도가 이보다 클 경우
window\_size = 5

for step in range(steps):
qc = build\_circuit(U)
qc = transpile(qc, sim, optimization\_level=0)
result = sim.run(qc, shots=shots, noise\_model=noise\_model).result()
counts = result.get\_counts()

```
# --- 엔트로피 및 편향도 계산 ---
probs = np.array(list(counts.values())) / shots
H = entropy(probs, base=2)
max_bias = max(probs)

entropy_traj.append(H)
bias_traj.append(max_bias)
entropy_window.append(H)
bias_window.append(max_bias)

# 조건부 구조 확장 판단
if len(entropy_window) > window_size:
    entropy_window.pop(0)
    bias_window.pop(0)
    dH = max(entropy_window) - min(entropy_window)
    avg_bias = np.mean(bias_window)
    if dH < entropy_threshold and avg_bias > bias_threshold:
        U.append(np.pi / 4)  # 구조 확장
        entropy_window = []
        bias_window = []

n_history.append(len(U))
```

# --- 시각화 ---

plt.figure(figsize=(10, 4))
plt.plot(n\_history, marker='o')
plt.title("Qubit Count Over Time (Conditional Expansion: Entropy + Bias)")
plt.xlabel("Step")
plt.ylabel("Qubit Count")
plt.grid(True)
plt.tight\_layout()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(entropy\_traj, label="Entropy", marker='o')
plt.plot(bias\_traj, label="Output Bias", marker='x')
plt.title("Entropy and Bias Over Feedback Steps")
plt.xlabel("Step")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.tight\_layout()
plt.show()


![Model B result](../01_papers/figures/result_2.png)